## Extraction of RMH 2018 texts from xml

In [50]:
# The location of RMH to read
rmh_dir = '/work/helgasvala/data/raw/risamalheild'

# The location of where to write the results
target_dir = '/work/inga/data/rmh'

In [51]:
from pathlib import Path

rmh_dir = Path(rmh_dir)
print(rmh_dir)
target_dir = Path(target_dir)
print(target_dir)
assert rmh_dir.exists()
if not target_dir.exists():
    target_dir.mkdir()

/work/helgasvala/data/raw/risamalheild
/work/inga/data/rmh


In [52]:
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from pathlib import Path
from typing import Sequence
from xml.etree import ElementTree as ET

def tei_read_file(path: Path) -> Sequence[str]:
    """Reads a tei file to extract the contents. Hand-tailored to reading the RMH.

    Adjusted code from xml_tools.py from Róbert Kjaran <robert@kjaran.com>

    :param path: A pathlib.Path to the .tei file to read.\n
    :return: A List of sentences as defined in the .tei file.
    """
    NS = {'a': 'http://www.tei-c.org/ns/1.0'}
    root = ET.parse(str(path)).getroot()
    sentences = []
    # We gather all the paragraphs from the body, avoiding the divs
    for paragraph_node in root.findall('.//a:body//a:p', NS):
        for sentence_node in paragraph_node.findall('.//a:s', NS):
            tokens = [(token_node.text, token_node.attrib['type'])
                      for token_node in sentence_node.findall('./*')]
            sentence: List[str] = []
            for i, token in enumerate(tokens):
                if token[0] is None:
                    continue
                sentence.append(token[0] + ' ')
            sentences.append(''.join(sentence).strip() + '\n')
    return sentences


def tei_read(paths: Sequence[Path], out_path: Path) -> bool:
    """Reads a sequence of Path of TEI files from RMH and writes to a single file.
    Uses multiple threads.

    :param paths: A Sequence of pathlib.Path of .tei files to read.\n
    :param out_path: pathlib.Path to write the contents to.\n
    :return: True if successful.
    """
    with out_path.open('w+') as f_out:
        with ProcessPoolExecutor(max_workers=14) as executor:
            results = tqdm(executor.map(
                tei_read_file,
                paths,
                chunksize=200),
                total=len(paths))
            for result in results:
                f_out.write(''.join(result))
    return True

In [69]:
# When choosing only from a few specific data sources
from glob import glob
import os
import random

dirsDepth2 = glob(f"{rmh_dir}/*/*/")
for d2 in dirsDepth2:
    parent = Path(d2).parent
    parent_base = os.path.basename(os.path.normpath(parent))
    name = os.path.basename(os.path.normpath(d2))
    if Path(d2).is_dir():
        xml_files = glob(f"{d2}/**/*.xml", recursive=True)

        outdir=Path(target_dir).joinpath(parent_base)
        Path(outdir).mkdir(parents=True, exist_ok=True)
        rmh_txt = Path(outdir).joinpath(f'{name}.txt')
        
        tei_read([Path(tei_file) for tei_file in xml_files], rmh_txt)

100%|██████████| 20449/20449 [01:21<00:00, 250.61it/s]
